*extract pollution data*



In [6]:
import pandas as pd
import datetime as dt
#  import numpy as np
import json
import requests
import os

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Set up directories
input_dir = 'input_data'
landing_dir = 'landing_data'
cleaned_dir = 'cleaned_data'

In [7]:
pol_input_file = os.path.join( input_dir, 'pollution_us_2000_2016.csv')

# pol_input_file = 'input_data/pollution_us_2000_2016.csv'

In [8]:
pol_df = pd.read_csv(pol_input_file)


In [9]:
# Drop unused columns 
pol_df.drop(columns=['State Code', 'County Code', 'Site Num', 'Address', 'County', 'City', \
                            'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', \
                            'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', \
                            'SO2 1st Max Value', 'CO 1st Max Hour', 'CO AQI'], inplace=True)

In [10]:
# df.rename(columns={"A": "a", "B": "c"})
pol_df.rename(columns={"Unnamed: 0": "id", "Date Local": "Date_Local", "NO2 Units": "NO2_Units", "NO2 Mean": "NO2_Mean", \
                  "O3 Units": "O3_Units", "O3 Mean": "O3_Mean", "SO2 Units": "SO2_Units", \
                  "SO2 Mean": "SO2_Mean", "CO Units": "CO_Units", "CO Mean": "CO_Mean" }, inplace=True)



In [11]:
pol_df.head()


,id,State,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value
0,0,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,1.145833,4.2
1,1,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,0.878947,2.2
2,2,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,2.975000,23,NaN,Parts per million,1.145833,4.2
3,3,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,2.975000,23,NaN,Parts per million,0.878947,2.2
4,4,Arizona,2000-01-02,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,22,4.0,Parts per million,0.850000,1.6


In [12]:
pol_df.dropna(inplace=True)

# df = df[df.line_race != 0]
pol_df = pol_df[pol_df.State != 'Country Of Mexico']

In [13]:
pol_df.head(20)


,id,State,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value
0,0,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,1.145833,4.2
1,1,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,0.878947,2.2
4,4,Arizona,2000-01-02,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,22,4.0,Parts per million,0.850000,1.6
5,5,Arizona,2000-01-02,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,22,4.0,Parts per million,1.066667,2.3
8,8,Arizona,2000-01-03,Parts per billion,38.125000,Parts per million,0.007958,Parts per billion,5.250000,19,16.0,Parts per million,1.929167,4.4
9,9,Arizona,2000-01-03,Parts per billion,38.125000,Parts per million,0.007958,Parts per billion,5.250000,19,16.0,Parts per million,1.762500,2.5
12,12,Arizona,2000-01-04,Parts per billion,40.260870,Parts per million,0.014167,Parts per billion,7.083333,8,23.0,Parts per million,1.991667,5.1
13,13,Arizona,2000-01-04,Parts per billion,40.260870,Parts per million,0.014167,Parts per billion,7.083333,8,23.0,Parts per million,1.829167,3.0
16,16,Arizona,2000-01-05,Parts per billion,48.450000,Parts per million,0.006667,Parts per billion,8.708333,7,21.0,Parts per million,2.620833,5.6
17,17,Arizona,2000-01-05,Parts per billion,48.450000,Parts per million,0.006667,Parts per billion,8.708333,7,21.0,Parts per million,2.700000,3.7


In [14]:
# # iterate/loop over the rows in the DF.  Split 'Date_local', that I assume
# # is MM/DD/YYYY,
# # on forward slash '/' take the third item Year and store in new list 'll_year'
# # then store this in the df as column 'year'
# # for row in df.iterrows():

# # for row in df.iterrows():  NOTE:  Itertion is not suggested trying something different
# #result = [f(x) for x in df['col']]
result = [x.split("-") for x in pol_df['Date_Local']]


In [15]:
# # unpack it
ll_yyyy,ll_mm, ll_dd = map(list, zip(*result))

In [16]:
pol_df['year'] = ll_yyyy
pol_df.head()

,id,State,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value,year
0,0,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,1.145833,4.2,2000
1,1,Arizona,2000-01-01,Parts per billion,19.041667,Parts per million,0.022500,Parts per billion,3.000000,21,13.0,Parts per million,0.878947,2.2,2000
4,4,Arizona,2000-01-02,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,22,4.0,Parts per million,0.850000,1.6,2000
5,5,Arizona,2000-01-02,Parts per billion,22.958333,Parts per million,0.013375,Parts per billion,1.958333,22,4.0,Parts per million,1.066667,2.3,2000
8,8,Arizona,2000-01-03,Parts per billion,38.125000,Parts per million,0.007958,Parts per billion,5.250000,19,16.0,Parts per million,1.929167,4.4,2000


In [17]:
#import data from url to get the state abbr
url='http://worldpopulationreview.com/static/states/name-abbr.json'
d = requests.get(url).json()
states_abbr = pd.DataFrame(d.items() , columns=["State", "State Abbr"])
states_abbr.head()
states_abbr.rename(columns={"State Abbr": "State_Abbr"}, inplace=True)

In [18]:
pol_updated_df = pd.merge(states_abbr, pol_df, \
                                how='left', on='State')
pol_updated_df.head()

,State,State_Abbr,id,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value,year
0,Alabama,AL,0.0,2013-12-01,Parts per billion,17.208333,Parts per million,0.013542,Parts per billion,0.313636,11.0,1.0,Parts per million,0.259091,0.80,2013
1,Alabama,AL,1.0,2013-12-01,Parts per billion,17.208333,Parts per million,0.013542,Parts per billion,0.313636,11.0,1.0,Parts per million,0.266667,0.50,2013
2,Alabama,AL,4.0,2013-12-02,Parts per billion,20.687500,Parts per million,0.009375,Parts per billion,0.530000,11.0,3.0,Parts per million,0.305625,0.47,2013
3,Alabama,AL,5.0,2013-12-02,Parts per billion,20.687500,Parts per million,0.009375,Parts per billion,0.530000,11.0,3.0,Parts per million,0.400000,0.50,2013
4,Alabama,AL,8.0,2013-12-03,Parts per billion,14.912500,Parts per million,0.008167,Parts per billion,0.305263,11.0,3.0,Parts per million,0.216818,0.35,2013


In [19]:
# # pd.to_numeric(s, downcast='integer')
pol_updated_df['year'] = pd.to_numeric(pol_updated_df['year'], downcast='integer')



pol_13_16_df = pol_updated_df.loc[pol_updated_df['year'] >= 2013]
pol_13_16_df.head()


,State,State_Abbr,id,Date_Local,NO2_Units,NO2_Mean,O3_Units,O3_Mean,SO2_Units,SO2_Mean,SO2 1st Max Hour,SO2 AQI,CO_Units,CO_Mean,CO 1st Max Value,year
0,Alabama,AL,0.0,2013-12-01,Parts per billion,17.208333,Parts per million,0.013542,Parts per billion,0.313636,11.0,1.0,Parts per million,0.259091,0.80,2013.0
1,Alabama,AL,1.0,2013-12-01,Parts per billion,17.208333,Parts per million,0.013542,Parts per billion,0.313636,11.0,1.0,Parts per million,0.266667,0.50,2013.0
2,Alabama,AL,4.0,2013-12-02,Parts per billion,20.687500,Parts per million,0.009375,Parts per billion,0.530000,11.0,3.0,Parts per million,0.305625,0.47,2013.0
3,Alabama,AL,5.0,2013-12-02,Parts per billion,20.687500,Parts per million,0.009375,Parts per billion,0.530000,11.0,3.0,Parts per million,0.400000,0.50,2013.0
4,Alabama,AL,8.0,2013-12-03,Parts per billion,14.912500,Parts per million,0.008167,Parts per billion,0.305263,11.0,3.0,Parts per million,0.216818,0.35,2013.0


# Store this in dir output for loading into the database.
pol_13_16_df.to_csv

In [21]:
output_file = os.path.join( landing_dir, 'pol_13_16_df.csv')

pol_13_16_df.to_csv(output_file, index=False)